In [1]:
import math
import cairo

In [2]:
DIM = 3000
DIM_POINTS = 590
CX = CY = 0.5

ANGLE_START = math.pi / 2
ANGLE_BY_5 = 2 * math.pi / 5
ANGLE_BY_10 = 2 * math.pi / 10
ANGLE_BY_100 = 2 * math.pi / 100

In [3]:
surface = cairo.RecordingSurface(cairo.Content.COLOR_ALPHA, 
                                 cairo.Rectangle(0, 0, DIM, DIM))
ctx = cairo.Context(surface)
ctx.scale(DIM, DIM)

In [4]:
ctx.set_source_rgb(0, 0, 0)

In [5]:
def circle(cx, cy, r, width = 10):
    ctx.save()
    ctx.set_line_width(width)
    ctx.arc(cx, cy, r, 0, math.pi * 2)
    ctx.stroke()
    ctx.restore()
    
def radial_line(cx, cy, start_rad, end_rad, angle, width = 10):
    x1 = cx + start_rad * math.cos(angle)
    y1 = cx + start_rad * math.sin(angle)
    x2 = cx + end_rad * math.cos(angle)
    y2 = cx + end_rad * math.sin(angle)
    
    ctx.save()
    ctx.set_line_width(width)
    ctx.move_to(x1, y1)
    ctx.line_to(x2, y2)
    ctx.stroke()
    ctx.restore()
    

In [6]:
thick_width = 0.004
medium_width = thick_width / 2
thin_width = thick_width / 5

size = 0.018
num_circles = 23
min_radius = 3 * size
max_radius = min_radius + (num_circles - 1) * size

r0 = min_radius
r10 = min_radius + 10 * size
r11 = min_radius + 11 * size
r21 = min_radius + 21 * size
r22 = min_radius + 22 * size

In [7]:
# Draw concentric circles
for i in range(num_circles):
    r = min_radius + i * size
    w = thick_width if i in [0, 10, 11, 21, 22] else thin_width
    circle(CX, CY, r, w)

In [8]:
# Level 1 sections
a = ANGLE_START
for i in range(0, 5):
    radial_line(CX, CY, min_radius, max_radius, a, thick_width)
    a += ANGLE_BY_5

# Level 2 sections
a = ANGLE_START
for i in range(0, 10):
    radial_line(CX, CY, r11, r21, a, medium_width)
    a += ANGLE_BY_10

# Level 3 sections
a = ANGLE_START
for i in range(0, 100):
    radial_line(CX, CY, r0, r10, a, thin_width)
    radial_line(CX, CY, r11, r21, a, thin_width)
    a += ANGLE_BY_100
    
    

In [9]:
def make_png_surface():
    global png_surface
    
    tmp_svg_surface = cairo.SVGSurface('tnp.svg', DIM, DIM)
    ctx = cairo.Context(tmp_svg_surface)
    scale = float(DIM) / float(DIM_POINTS)
    ctx.scale(scale, scale)
    ctx.set_source_surface(svg_surface)
    ctx.rectangle(0, 0, DIM - 1, DIM - 1)
    ctx.fill()
    
    png_surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, DIM, DIM)
    ctx = cairo.Context(png_surface)
    ctx.set_source_surface(tmp_svg_surface)
    ctx.rectangle(0, 0, DIM - 1, DIM - 1)
    ctx.fill()

def render_surface(src, src_dim, dst, dst_dim):
    ctx = cairo.Context(dst)
    scale = float(dst_dim) / float(src_dim)
    ctx.scale(scale, scale)
    ctx.set_source_surface(src)
    ctx.rectangle(0, 0, dst_dim - 1, dst_dim - 1)
    ctx.paint()
    
svg_surface = cairo.SVGSurface('copic_color_wheel.svg', 
                               DIM_POINTS, DIM_POINTS)
render_surface(surface, DIM, svg_surface, DIM_POINTS)

png_surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, DIM, DIM)
render_surface(surface, DIM, png_surface, DIM)
png_surface.write_to_png('copic_color_wheel.png')

#ctx.set_source_rgb(1, 1, 1)
#ctx.rectangle(0, 0, 1, 1)
#ctx.fill()



In [10]:
png_surface.finish()
svg_surface.finish()